In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data.csv')

AI classes assignment

In [ ]:
eng_cylinders_median = df['Engine Cylinders'].median()
eng_cylinders_mode = df['Engine Cylinders'].mode()
df['Engine Cylinders'].fillna(value=eng_cylinders_mode, inplace=True)
eng_cylinders_median2 = df['Engine Cylinders'].median()

print(eng_cylinders_median, eng_cylinders_median2)
df['Engine Cylinders'].isna().sum()


In [ ]:
lotusdf = df[df['Make'] == 'Lotus']
lotusdf = lotusdf[['Engine HP' ,'Engine Cylinders']]
lotusdf = lotusdf.drop_duplicates()
lotusdf

In [ ]:
#converting lotus df to numpy array
x = lotusdf.to_numpy()
#transpose of x  ("@" operation is multiplication)
x_t = x.T @ x
#inverse x transpose
x_t_inverse = np.linalg.inv(x_t)
x_t_inverse


In [ ]:
y = np.array([1100, 800, 750, 850, 1300, 1000, 1000, 1300, 800])

w = (x_t_inverse @ x.T) @ y

w[0]

#w is the normal equation
#NORMAL EQUATION => 
# w = (X.T * X)^-1 * X.T * Y

My Puny attempt at EDA (PS I have no idea what im doing Im just going to play around here)

In [ ]:
#check for missing values
df.isna().sum()

In [ ]:
#aiit a couple of missing values now to find a way to fill them 
#I could try filling the values, column by column ()

#engine fuel type is a categorical column 
print(df['Engine Fuel Type'].unique())      #10 unique categories

df[df['Engine Fuel Type'].isna()]       #okayy we can see them they probably have the same fuel type since they are of the same car model
#sooo i check the data manually and the fuel type is most likely going to be 'regular unleaded' since the later model of the car uses the same fuel type
#I'm just going to perform a simple ffill

df['Engine Fuel Type'].fillna(method='ffill', inplace=True)

In [ ]:
#Engine HP
#since Engine HP is a numeric column
#we can check distribution and fill columns from the graph

print(df['Engine HP'].mean())
print(df['Engine HP'].median())
print(df['Engine HP'].mode())
sns.kdeplot(df['Engine HP'])
#sns.boxplot(df['Engine HP'])

#I'm going to use median since the distribution is skewed and there are outliers so we cannot really use mean
#I would've probably used mode but !!! It just does'nt fell right to do so 🤔

df['Engine HP'].fillna(df['Engine HP'].median(), inplace = True)

#sns.kdeplot(df['Engine HP'])    #mehh looks roughly thesame idk

In [ ]:
#Engine Cylinders
#using the same method sha

print(df['Engine Cylinders'].mean())
print(df['Engine Cylinders'].median())
print(df['Engine Cylinders'].mode())
sns.boxplot(df['Engine Cylinders'])

#im using mean.. dont ask why... 'cause I don't know (for now at least)
df['Engine Cylinders'].fillna(df['Engine Cylinders'].median(), inplace = True)

sns.boxplot(df['Engine Cylinders'])

In [ ]:
#I dont care about the number of doors just use ffill
df['Number of Doors'].fillna(method='ffill', inplace = True)


In [ ]:
#Market Category
#this is a categorical column
lst = df['Market Category'].unique()
print(lst)

OKAY THE COLUMN LABELS WERE TOO MUCH (AND ALMOST OVERWHELMING) SO I DECIDED TO CREATE A MODEL TO PREDICT THEM 

In [ ]:
print(len(lst))

In [ ]:
category_list = []
for i in lst:
    if isinstance(i, str):
        if ',' in i:
            new_list = i.split(',')
            for j in new_list:
                if j in category_list:
                    print(j, 'already found')
                    continue
                else:
                    category_list.append(j)
                    print(j, 'added')
        else:
            if i in category_list:
                print(i, 'already found')
                continue
            else:
                category_list.append(i)
                print(i, 'added')


In [ ]:
category_list

In [ ]:
prediction_df = df.loc[df['Market Category'].isna()]
df.drop(df.index[df['Market Category'].isna()], inplace=True)


In [ ]:
prediction_df

In [ ]:
#well then our data contains one label combination than others 
plt.figure(figsize=(10, 10))
plt.xticks(rotation=-90)
sns.barplot(x=df['Market Category'].unique(), y=df['Market Category'].value_counts())

In [ ]:
#seperating them
df_y = df.pop('Market Category')

In [ ]:
for col in df.columns:
    print(df[col].unique())
    print('\n')

In [ ]:
df_y.apply(lambda x: [i for i in x])
df.pop('Model')
prediction_df.pop('Model')
prediction_df.pop('Market Category')


In [ ]:
#custom OHE function for list

def ohe_list(df_col, categories):
    #create a dictionary map
    label_to_int = dict((c, i) for i, c in enumerate(categories))
    int_to_label = dict((i, c) for i, c in enumerate(categories))

    #encode to integer
    label_encoded = [[[label_to_int[label] for label in cell.split(',') ] for cell in row] for row in df_col]

    #create one hot list
    oh_list = list()

    for row in label_encoded:
        for cell in row:
            cell_enc = [0 for _ in range(len(categories))]
            for label in cell:
                cell_enc[label] = 1
            oh_list.append(cell_enc)

    return oh_list 


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score

def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]

    #evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

    for train_ix, test_ix in cv.split(X):
        #prep training data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]

        #model
        model = get_model(n_inputs, n_outputs)
        model.fit(X_train, y_train, epochs=100)

        y_hat = model.predict(y_test)
        y_hat = y_hat.round()

        acc = accuracy_score(y_test, y_hat)

        print('>%.3f' % acc)
        results.append(acc)
    return results